# Arrays and Vectors

As noted in the lecture, arrays are not vectors, two dimensional arrays are not matrices, *etc*. Arrays are far more general objects than vectors or matrices, yet arrays can be (and are) used to represent vectors and matrices. Here will explore some of the implications of this and some of the care that must be taken when using a general object (array) to represent specific objects (vectors and matrices).

## Initialization

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
# Globally fix plot styling
import matplotlib as mpl
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)

# For random numbers
rng = np.random.default_rng()

We have some familiarity with arrays since we have already been using them. For linear algebra the main routines are contained in `scipy.linalg`. Behind the scenes the calculations are done using `BLAS` and `LAPACK` libraries. Depending on how your version was compiled these can be highly optimized and even parallelized.

*Notes:*
1. Some functions *also* have implementations in NumPy. In general we are going to prefer the versions from `scipy.linalg`. In practice, both versions should give the same results except in extreme cases, such as dealing with (numerically) singular matrices.
2. Since we will represent both vectors and matrices using NumPy arrays we will need to be careful. For example, we know what multiplying a matrix and a vector means, whereas multiplying a two dimensional array and a one dimensional array is not uniquely defined. There *is* a matrix object in NumPy but it is not well developed nor well supported.  In fact, it is deprecated and will be removed from future versions. It should never be used! Due to the fact that we are using arrays instead of matrices this means we will need to use a special function, `np.dot`, or a special operator, `@`, when we multiply a matrix and a vector. (We will mostly discuss the `np.dot` function below and save the `@` operator for the future.)

As always, begin by looking at the documentation. From `scipy.linalg` we see there are many routines. Here we will focus on some of those from "Basics". We will encounter some of the other functions in future weeks.

In [ ]:
la?

As usual there are many things defined in this module. For our purposes here we are interested in the basic functionality, in particular calculating inverses using `la.inv()`. There are many other specialized functions, many matrix decompositions, *etc*., all of which have their particular uses.

In [ ]:
la.inv?

Notice that this calculates the inverse for a **square matrix**. There are options for optimizing the function that are common to many of the functions in `scipy.linalg`. These are particularly useful when dealing with large matrices when we want to avoid making copies. By default these functions do make a copy and do not overwrite their inputs. This is a good default, but can lead to slower, more memory intensive code. For all of our uses this will not be important so we will just use the defaults.

As one test we can verify that `la.inv()` really does calculate an inverse. Recall that for a square matrix with $\det(\mathsf{A})\ne 0$ there exists an inverse, $\mathsf{A}^{-1}$ that satisfies
$$ \mathsf{A} \mathsf{A}^{-1} = \mathsf{A}^{-1} \mathsf{A} = \mathsf{1}. $$

In [2]:
# Construct a random matrix
A = rng.random(size=(3, 3))
# Calculate its inverse
Ainv = la.inv(A)
# Print some results
print(f"""
det(A) = {la.det(A)}
A Ainv = {np.dot(A, Ainv)}
Ainv A = {np.dot(Ainv, A)}""")


det(A) = -0.023981625997744314
A Ainv = [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.55111512e-17  1.00000000e+00 -1.11022302e-16]
 [ 2.22044605e-16  0.00000000e+00  1.00000000e+00]]
Ainv A = [[ 1.00000000e+00  1.11022302e-16 -4.16333634e-17]
 [ 2.22044605e-16  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


There are few things to notice here.  First, we used `np.dot()` to multiply the matrices, we **did not use** `A*Ainv`. Why not?

Let us compare the two options.

In [3]:
print(f"""
np.dot(A, Ainv) = {np.dot(A, Ainv)}
A * Ainv = {A * Ainv}""")


np.dot(A, Ainv) = [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.55111512e-17  1.00000000e+00 -1.11022302e-16]
 [ 2.22044605e-16  0.00000000e+00  1.00000000e+00]]
A * Ainv = [[ -0.12102962   3.43231602  -0.05580331]
 [  0.67481528   0.65504712  -0.06345617]
 [ -2.65656306 -21.76892649   2.58892492]]


The first case gives the identity matrix (or something close to it) but what do we get in the second case? The function `np.dot()` does *matrix multiplication* :
$$ (\mathsf{A} \mathsf{A}^{-1})_{ij} = \sum_{k} A_{ik} (A^{-1})_{kj}. $$

The usual multiplication, `A*Ainv`, multiplies the components of the two arrays
$$ (\mathsf{A} * \mathsf{A}^{-1})_{ij} = A_{ij} * (A^{-1})_{ij}. $$

In both cases we end up with a two dimensional array, but they are very different arrays!

The next thing to notice is that we are suppose to get the identity matrix. We can construct this in NumPy in a few ways, one is using `np.eye()`.

In [4]:
np.eye(A.shape[0])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

Do our matrix multiplications actually return the identity matrix? We see that it is not exactly the identity matrix, but we do not expect it to be. (Why not?) For small arrays like we have here we can look at all the components and see they the diagonal entries are one and the off diagonal entries are close enough to zero, as expected. For larger arrays we can once again use `np.allclose`.

As the documentation describes, this will test each component of our arrays testing if they are "close" to each other in both an absolute and relative sense. In other words, there are two tolerances. The formula described in the documentation is, for components $a$ and $b$ of the two arrays, respectively, we say these values are close if
$$ |a - b| \le (a_{\mathrm{tol}} + r_{\mathrm{tol}} |b|), $$
where $a_{\mathrm{tol}}$ is the absolute tolerance and $r_{\mathrm{tol}}$ is the relative tolerance. As the documentation also notes, this definition is asymmetric, $a$ and $b$ are treated differently. This is an unfortunate choice and something that may be corrected in the future.

For our purposes we can use this to verify the inverse found above

In [5]:
print(f"""
A Ainv = 1? {np.allclose(np.eye(A.shape[0]), np.dot(A, Ainv))}
Ainv A = 1? {np.allclose(np.eye(A.shape[0]), np.dot(Ainv, A))}
""")


A Ainv = 1? True
Ainv A = 1? True



### Lists versus Arrays

The basic data structures in Python itself are lists and tuples. We can think of a tuple as a read only list. The basic data structure in NumPy is an array. Arrays and lists are not the same thing, even though most NumPy and SciPy functions will treat them as the same. In fact, many of us have "gotten away with" using a list where we meant an array, since the functions we used silently converted lists to arrays.

To see that there is a difference we first construct the two different types of objects

In [6]:
a_list = [1., 2, 3]
# We can convert the list to an array
a_arr = np.array(a_list)
print(f"""List : {a_list}
Array: {a_arr}""")

List : [1.0, 2, 3]
Array: [1. 2. 3.]


Notice that these are different! The list preserves the types of the objects we put into it. The first element is a float and the last two are integers. The array, on the other hand, converts everything to the same (largest) type, so everything is a float.  Python lists can contain different types of elements. NumPy arrays must have all the same type.

Next, if we try to add a float to a list or an array the addition works fine for the array but gives an error for the list.

In [7]:
print("Array sum :", a_arr + 1.1)
print("List sum  :", a_list + 1.1)

Array sum : [2.1 3.1 4.1]


TypeError: can only concatenate list (not "float") to list

Notice that it did try to interpret the addition in some way, it just could not make sense of the operation.

On the other hand, if we multiplied by an integer ...!

In [8]:
print("Array mult :", a_arr * 2)
print("List mult  :", a_list * 2)

Array mult : [2. 4. 6.]
List mult  : [1.0, 2, 3, 1.0, 2, 3]


The summary of this is that we must be careful with how we represent our information. If we mean to use arrays then we should use arrays. Even if the functions we are using silently "fix" this for us, we always should use the correct data structure. If we do not, then we will run into a problem, eventually.

### Broadcasting

Another extremely powerful feature of arrays is broadcasting. When dealing with matrices and vectors an expression such as $\mathsf{A} + \vec v$ makes no sense. It is complete nonsense! However, when dealing with arrays we can define what this means. Similarly, we know that the multiplication of a matrix and a vector, $\mathsf{A}\vec v$, does make sense and produces a vector. So what does the multiplication of a two dimensional array and a one dimensional array produce?

The answer to all questions relating to the combination of different dimensional arrays is broadcasting. There are many powerful things that can be done with broadcasting, here we will just understand the basics. Though we will list some rules, the best approach, as always, is to **test your operations**. There is no need to guess at how things will behave, we can easily check!

**Basic Rule**

For array `A` with one more dimension than array `v`, broadcasting will "repeat" `v` along the missing dimension of `A`. For the two dimensional case this means if `A` is a $M\times N$ array, then `v` must be of length $N$ and broadcasting will "repeat" it $M$ times and operate on each row of `A` using the same `v`.

This can be phrased in more sophisticated ways, but it is easiest to see in an example. Here we create an array with 3 columns (notice the use of `reshape`).

In [9]:
A = np.arange(6).reshape((-1, 3))
v = np.arange(3) + 1
print("A =", A)
print("v =", v)

A = [[0 1 2]
 [3 4 5]]
v = [1 2 3]


We can add and multiply these two. Again notice that the operations are performed on each component of `A` using the same components of `v` for each row. More explicitly, for the case of addition:
$$ (A+v)_{ij} = A_{ij} + v_j. $$
A similar definition applies for multiplication (and it would also for subtraction, division, and other operators).

In [10]:
print("A+v =", A + v)
print("A*v =", A * v)

A+v = [[1 3 5]
 [4 6 8]]
A*v = [[ 0  2  6]
 [ 3  8 15]]


The key to broadcasting is that the shape of `v` must be the same shape as `A`, in all of its final dimensions. In the two dimensional case this means if `A` is $N\times M$ and `v` is of length $N$ then **broadcasting will not work** (at least not for $N\ne M$).

We can see that here.

In [11]:
A = np.arange(6).reshape((3, -1))
v = np.arange(3) + 1
print("A =", A)
print("v =", v)
print("A+v =", A + v)

A = [[0 1]
 [2 3]
 [4 5]]
v = [1 2 3]


ValueError: operands could not be broadcast together with shapes (3,2) (3,) 

Notice the error even tells us the broadcasting fails, and why. How can we fix this? Well, what we wanted to happen was to have `v` added to the *columns* of `A`, not the rows. This can easily be accomplished by taking the transpose of `A`. Since taking the transpose is so common there is shorthand for doing this, `A.T`. Here, if we take the transpose of `A` we can add it to `v` using the usual broadcasting. This is great, except the answer we get will have the rows and columns swapped. How do we fix this? Just take the transpose of the result!

In [12]:
print("Corrected A+v =", (A.T + v).T)

Corrected A+v = [[1 2]
 [4 5]
 [7 8]]


*Technical details:* The way this, and most slicing, is done is using what is called *strides*. It really just changes the way the array is accessed. No changes are made to the actual order of data in the array. It is not necessary to know this to use broadcasting, but it does mean that it not computationally expensive to use. It is implemented in an efficient manner.

## Newton-Raphson Method

The solution to a system of linear equations is something we will discuss next week. For a system of non-linear equations, or really higher dimensional root finding, is a large topic. Here we will present one, simple approach. There are, of course, many algorithms. The one we will use is the Newton-Raphson method which relies on knowing (or estimating) the derivatives of the functions involved.

To understand the technique we go back to our favorite topic from calculus, the Taylor series. Here we expand in more than one dimension. First consider a single function $f_i(\vec y)$. Here we expand
$$ f_i(\vec y + \Delta\vec y) = f_i(\vec y) + \sum_{j=1}^{n} \left. \frac{\partial f_i}{\partial y_j} \right|_{\Delta \vec y=0} \Delta y_j + \frac{1}{2} \sum_{j,k} \left( \frac{\partial f_i}{\partial y_j} \frac{\partial f_i}{\partial y_k} \right|_{\Delta \vec y=0} \Delta y_j \Delta y_k + \cdots, $$
where we the expansion is done about $\Delta\vec y=0$.

For our case we truncate the expansion, only keeping the first two terms. Then, for a system of functions (i.e., more than one function) we can write in matrix and vector notation the truncated expansion as
$$ \vec f(\vec y + \Delta\vec y) = \vec f(\vec y) + \mathsf{J}_{\vec f}(\vec y) \Delta\vec y, $$
where $\mathsf{J}$ is the Jacobian matrix with components
$$ J_{ij}(\vec y) = \frac{\partial f_i(\vec y)}{\partial y_j}. $$
If we set $\vec f(\vec y + \Delta\vec y) = 0$ we can solve to find $\Delta\vec y = - \mathsf{J}_{\vec f}^{-1}(\vec y) \vec f(\vec y)$.

The Newton-Raphson method is then to iterate on this. Let $\vec y_n$ be a guess for the desired solution. An improved guess is thus given by
$$ \vec y_{n+1} = \vec y_n + \Delta\vec y = \vec y_n - \mathsf{J}_{\vec f}^{-1}(\vec y_n) \vec f(\vec y_n). $$
From an initial guess, $\vec y_0$, we iteratively improve our estimate and stop when convergence has been obtained, that is, when the correction becomes sufficiently small.

**Warning:** We are going to use the algorithm as written and numerically calculate the inverse of the Jacobian matrix. In practice this is a bad approach. We should instead treat this as solving a system of linear equations; a topic we will cover next week.

### Jacobian Matrix

As we have seen, the Jacobian matrix is the matrix of first derivatives of the function. Written more explicitly
$$
\mathsf{J}_{\vec f} (\vec y) = \begin{pmatrix}
\frac{\partial f_1}{\partial y_1} & \frac{\partial f_1}{\partial y_2} & \cdots \\
\frac{\partial f_2}{\partial y_1} & \frac{\partial f_2}{\partial y_2} & \cdots \\
\vdots & \vdots & \ddots
\end{pmatrix}
$$

This matrix can be calculated analytically or numerically.

### Non-linear System

As a sample for solving a non-linear system using the Newton-Raphson method consider the equations
$$
\begin{align}
16 x_1^4 + 16 x_2^4 + x_3^4 &= 16 \\
x_1^2 + x_2^2 + x_3^2 &= 3 \\
x_1^3 - x_2 &= 0
\end{align}.
$$

To evaluate this numerically we first calculate the Jacobian analytically giving
$$ \mathsf{J} =
\begin{pmatrix}
64 x_1^3 & 64 x_2^3 & 4 x_3^3 \\
2 x_1 & 2 x_2 & 2 x_3 \\
3 x_1^2 & -1 & 0
\end{pmatrix}.
$$

Next we implement the system of functions and the Jacobian as Python functions which return NumPy arrays.

In [13]:
def funcs(y):
    """The system of non-linear equations.
    Here we assume y contains the variables (x1, x2, x3).
    We return the 3 equations in the ordered listed above.
    """
    f = np.zeros_like(y)
    f[0] = 16*y[0]**4 + 16*y[1]**4 + y[2]**4 - 16
    f[1] = y[0]**2 + y[1]**2 + y[2]**2 - 3
    f[2] = y[0]**3 - y[1]
    return f

def Jacobian(y):
    """Same as funcs but now returns the Jacobian."""
    J = np.zeros((len(y), len(y)))
    J[0,0] = 64*y[0]**3
    J[0,1] = 64*y[1]**3
    J[0,2] = 4*y[2]**3
    J[1,0] = 2*y[0]
    J[1,1] = 2*y[1]
    J[1,2] = 2*y[2]
    J[2,0] = 3*y[0]**2
    J[2,1] = -1
    J[2,2] = 0
    return J

To implement the Newton-Raphson method we write a function. The method does not care what system of equations is provided. All it needs is to know how to evaluate the function and the Jacobian. We stop when convergence is attained. As with any routine that relies on convergence we should guard against never converging! Thus we have a maximum number of iterations allowed. We return the number of iterations required. If this number is the same as the maximum then we do not know that we have converged.

The implementation thus requires a function, `f`, that returns the value of the system of equations for which we are finding the root, the Jacobian, `J`, an initial guess `y0`, and the maximum number of allowed iterations, `itermax`. Returned is the best guess of the root, `y`, and the number of iterations performed, `niter`. These are returned as a tuple.

In [14]:
def newton_raphson(f, Jacobian, y0, itermax):
    """Simplified function to solve a system of equations,
    f(y)=0,
    iterating from an initial guess, y0, using the Newton-Raphson method.
    Inputs:
      f : function that returns an array of N values.
          It is called as f(y)
      Jacobian : function that returns the NxN Jacobian for the system of
                 equations in f as a 2d array.
                 It is called as Jacobian(y)
      y0 : numpy array of size N containing the initial guess for the solution.
      itermax: integer, the maximum number of allowed iterations.
    Outputs: Tuple containing the following
      y : array of size N of the best estimate of the solution.
      niter : integer, the number of iterations required to find the solution.
    """
    # Copy y0 as our starting guess. We do not want to overwrite it.
    y = y0.copy()
    niter = 1
    while niter <= itermax:
        J = Jacobian(y)
        dy = np.dot(la.inv(J), f(y))
        y -= dy
        if np.allclose(np.zeros_like(dy), dy, atol=1e-10, rtol=1e-10):
            break
        niter += 1
    return (y, niter)

As an initial guess we use all ones.

In [15]:
y0 = np.ones(3)
(y, niter) = newton_raphson(funcs, Jacobian, y0, 20)
print(f"Solution = {y} after {niter} iterations")

Solution = [0.87796576 0.67675697 1.33085541] after 6 iterations


We can verify that this really is a solutions

In [16]:
print("f(y) must be zero: f(y) =", funcs(y))

f(y) must be zero: f(y) = [0. 0. 0.]


Or instead we write a test using `assert` that will report nothing if everything is fine and raise an error if the two arrays disagree.

In [17]:
assert(np.allclose(funcs(y), np.zeros_like(y)))

There are many things that could go wrong. Suppose we try to pass in a Python list instead of a NumPy array, does this work?

In [18]:
y0 = [1, 1, 1]
(y, niter) = newton_raphson(funcs, Jacobian, y0, 20)
print(f"Solution = {y} after {niter} iterations")

Solution = [0.87796576 0.67675697 1.33085541] after 6 iterations


Surprisingly it actually does! Why? It turns out arrays are smarter than we might have expected. In particular, NumPy "magically" handled subtracting a NumPy array from a Python list and turned the result into a NumPy array! We got lucky here, we should **not** rely on this behavior.

Trying to correct this suppose we convert `y0` to an array ourselves.

In [19]:
y0 = np.array([1, 1, 1])
(y, niter) = newton_raphson(funcs, Jacobian, y0, 20)
print(f"Solution = {y} after {niter} iterations")

UFuncTypeError: Cannot cast ufunc 'subtract' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'

Now it fails! The error explains why. We constructed an array of integers for `y0`. By copying this we tried to treat the solution as an array of integers, but this failed when we used `y -= dy` since this tried to subtract floating point numbers from integers and stick them back into an integer. If we had instead written `y = y - dy` it would have worked since `y` would have gotten replaced by an array of floats.

The summary of all of this is that we should be careful if we want to avoid surprises. If we want an array of floats then just make an array of floats! Here we should have used
```
y0 = np.array([1., 1, 1])
```
where any of the "1" should have been "1.". If even one of them is a floating point number then the whole array is made to be floating point numbers.

In [20]:
y0 = np.array([1., 1, 1])
print(y0)
print(y0.dtype)

[1. 1. 1.]
float64
